# バックテスト（過去検証）

このノートブックでは、過去のデータを使って投資戦略をテストする方法を学びます。

## バックテストとは？

バックテストは、過去のデータを使って投資戦略の有効性を検証することです。
- 戦略が過去にどの程度うまくいったかを確認
- リスクとリターンを定量的に評価
- 実際に投資する前に戦略を検証

## 目次
1. 環境設定
2. シンプルな移動平均戦略
3. バックテストの実行
4. パフォーマンス評価
5. 戦略の改善

## 1. 環境設定

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from decimal import Decimal

# 認証情報の設定
os.environ['JQUANTS_MAIL_ADDRESS'] = 'your_email@example.com'
os.environ['JQUANTS_PASSWORD'] = 'your_password'

import pyjquants as pjq
print('準備完了')

## 2. シンプルな移動平均戦略

### 戦略の説明

ゴールデンクロス・デッドクロス戦略：
- **買いシグナル**: 短期移動平均が長期移動平均を上抜け（ゴールデンクロス）
- **売りシグナル**: 短期移動平均が長期移動平均を下抜け（デッドクロス）

In [ ]:
# テスト対象の銘柄
stock = pjq.Stock('7203')  # トヨタ
print(f'銘柄: {stock.name} ({stock.code})')

# 株価データを取得
prices = stock.prices.copy()
print(f'データ期間: {prices["date"].min()} ~ {prices["date"].max()}')
print(f'データ件数: {len(prices)}日')

In [ ]:
# 移動平均を計算
short_window = 5   # 短期: 5日
long_window = 20   # 長期: 20日

prices['SMA_short'] = prices['close'].rolling(window=short_window).mean()
prices['SMA_long'] = prices['close'].rolling(window=long_window).mean()

# シグナルを生成
prices['signal'] = 0
prices.loc[prices['SMA_short'] > prices['SMA_long'], 'signal'] = 1   # 買い
prices.loc[prices['SMA_short'] <= prices['SMA_long'], 'signal'] = -1  # 売り

# シグナルの変化点を検出
prices['position'] = prices['signal'].diff()

print('シグナル計算完了')
prices[['date', 'close', 'SMA_short', 'SMA_long', 'signal', 'position']].tail(10)

### 移動平均とシグナルの可視化

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

# 株価と移動平均
ax.plot(prices['date'], prices['close'], label='Close', linewidth=1.5)
ax.plot(prices['date'], prices['SMA_short'], label=f'SMA {short_window}', linewidth=1)
ax.plot(prices['date'], prices['SMA_long'], label=f'SMA {long_window}', linewidth=1)

# 買いシグナル（ゴールデンクロス）
buy_signals = prices[prices['position'] == 2]
ax.scatter(buy_signals['date'], buy_signals['close'], 
           marker='^', color='green', s=100, label='Buy Signal', zorder=5)

# 売りシグナル（デッドクロス）
sell_signals = prices[prices['position'] == -2]
ax.scatter(sell_signals['date'], sell_signals['close'], 
           marker='v', color='red', s=100, label='Sell Signal', zorder=5)

ax.set_title(f'{stock.name} - Moving Average Crossover Strategy', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Price (JPY)')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. バックテストの実行

トレーダーを使って実際にシミュレーションします。

In [ ]:
# バックテスト用のトレーダーを作成
initial_cash = 1_000_000  # 100万円
trader = pjq.Trader(initial_cash=initial_cash)

# 取引単位
trade_quantity = 100  # 100株単位

# 有効なデータのみ使用（移動平均が計算できる期間）
valid_prices = prices.dropna().copy()

# バックテスト実行
trades = []
holdings = 0

for idx, row in valid_prices.iterrows():
    trade_date = row['date']
    
    # ゴールデンクロス（買い）
    if row['position'] == 2 and holdings == 0:
        order = trader.buy(stock, trade_quantity)
        executions = trader.simulate_fills(trade_date)
        if executions:
            holdings = trade_quantity
            trades.append({
                'date': trade_date,
                'action': 'BUY',
                'price': float(executions[0].price),
                'quantity': trade_quantity
            })
    
    # デッドクロス（売り）
    elif row['position'] == -2 and holdings > 0:
        order = trader.sell(stock, holdings)
        executions = trader.simulate_fills(trade_date)
        if executions:
            trades.append({
                'date': trade_date,
                'action': 'SELL',
                'price': float(executions[0].price),
                'quantity': holdings
            })
            holdings = 0

print(f'取引回数: {len(trades)}回')
pd.DataFrame(trades)

## 4. パフォーマンス評価

In [ ]:
# 最終評価額
final_value = float(trader.portfolio.total_value)
total_return = (final_value - initial_cash) / initial_cash * 100

print('=' * 50)
print('バックテスト結果')
print('=' * 50)
print(f'初期資金:     {initial_cash:>12,}円')
print(f'最終評価額:   {final_value:>12,.0f}円')
print(f'損益:         {final_value - initial_cash:>12,.0f}円')
print(f'リターン:     {total_return:>12.2f}%')
print(f'取引回数:     {len(trades):>12}回')
print('=' * 50)

In [ ]:
# Buy & Hold戦略との比較
first_price = float(valid_prices.iloc[0]['close'])
last_price = float(valid_prices.iloc[-1]['close'])

# Buy & Holdで買える株数
bh_shares = int(initial_cash / first_price)
bh_final_value = bh_shares * last_price + (initial_cash - bh_shares * first_price)
bh_return = (bh_final_value - initial_cash) / initial_cash * 100

print('\n戦略比較')
print('-' * 50)
print(f'移動平均戦略:  {total_return:>8.2f}%')
print(f'Buy & Hold:    {bh_return:>8.2f}%')
print(f'差分:          {total_return - bh_return:>8.2f}%')
print('-' * 50)

## 5. 戦略の改善

### パラメータの調整

In [ ]:
def backtest_ma_strategy(prices_df, short_window, long_window, initial_cash=1_000_000):
    """移動平均戦略のバックテストを実行"""
    df = prices_df.copy()
    
    # 移動平均を計算
    df['SMA_short'] = df['close'].rolling(window=short_window).mean()
    df['SMA_long'] = df['close'].rolling(window=long_window).mean()
    
    # シグナルを生成
    df['signal'] = 0
    df.loc[df['SMA_short'] > df['SMA_long'], 'signal'] = 1
    df.loc[df['SMA_short'] <= df['SMA_long'], 'signal'] = -1
    df['position'] = df['signal'].diff()
    
    # シンプルなリターン計算
    df = df.dropna()
    df['daily_return'] = df['close'].pct_change()
    df['strategy_return'] = df['daily_return'] * df['signal'].shift(1)
    
    total_return = (1 + df['strategy_return'].fillna(0)).prod() - 1
    
    return total_return * 100

# 異なるパラメータでテスト
results = []
for short in [3, 5, 7, 10]:
    for long in [15, 20, 25, 30]:
        if short < long:
            ret = backtest_ma_strategy(prices, short, long)
            results.append({
                'short': short,
                'long': long,
                'return': ret
            })

results_df = pd.DataFrame(results).sort_values('return', ascending=False)
print('パラメータ別リターン（上位5件）:')
results_df.head()

## まとめ

このノートブックでは以下を学びました：

- **移動平均戦略**: ゴールデンクロス・デッドクロスの実装
- **バックテスト**: 過去データでの戦略検証
- **パフォーマンス評価**: リターンの計算と比較
- **パラメータ最適化**: 複数パラメータのテスト

### 注意点

- 過去のパフォーマンスは将来を保証しません
- 取引コスト（手数料、スリッページ）は考慮していません
- 過度のパラメータ最適化は過学習のリスクがあります

---

**免責事項**: このノートブックは教育目的であり、投資助言ではありません。